# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [14]:
import pandas as pd
import sqlite3

conn =sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [15]:
pd.read_sql("""

SELECT firstName, lastName
  FROM employees AS e
  JOIN offices AS o
      USING(officeCode)
  WHERE city = 'Boston'


""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [16]:
pd.read_sql("""

SELECT o.officeCode, o.city, COUNT(e.EmployeeNumber) AS num_emps 
  FROM offices AS o
  LEFT JOIN employees AS e
  USING(officeCode)
      GROUP BY officeCode
  HAVING num_emps = 0


""", conn)

,officeCode,city,num_emps
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [18]:
"""
How many customers does each office have?
"""

pd.read_sql("""

SELECT o.officeCode, o.city, COUNT(c.customerNumber) AS num_customers 
  FROM offices AS o
  JOIN employees AS e
      USING(officeCode)
  JOIN customer AS c
      ON e.employeeNumber = c.salesRepEmployeeNumber
  GROUP BY officeCode


""", conn)

,officeCode,city,num_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [24]:
"""
What product has the highest quantity ordered in an order? 
"""

pd.read_sql("""

SELECT p.productName, MAX(ord.quantityOrdered) AS num_ordered
  FROM products AS p
  JOIN orderdetails AS ord
      USING(productCode)


""", conn)

,productName,num_ordered
0,1969 Dodge Charger,97


In [27]:
"""
Show the country and states with the top 10 payment amounts
"""

pd.read_sql("""

SELECT cust.country, cust.state, pay.amount
  FROM customers AS cust
  JOIN payments AS pay
      USING(customerNumber)
  ORDER BY pay.amount DESC


""", conn)

,country,state,amount
0,Spain,,120166.58
1,Spain,,116208.40
2,USA,CA,111654.40
3,Singapore,,105743.00
4,USA,CA,101244.59
...,...,...,...
268,France,,1676.14
269,Belgium,,1627.56
270,Norway,,1491.38
271,Belgium,,1128.20


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [29]:
pd.read_sql("""

SELECT emp.firstName, emp.lastName, productName
    FROM employees AS emp
    JOIN customers AS c
    ON emp.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING(customerNumber)
    JOIN orderdetails
        USING(orderNumber)
    JOIN products
        USING(productCode)
    

""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [33]:
pd.read_sql("""

SELECT emp.firstName, emp.lastName, SUM(quantityOrdered) AS products_sold
    FROM employees AS emp
    JOIN customers AS c
    ON emp.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING(customerNumber)
    JOIN orderdetails
        USING(orderNumber)
    GROUP BY firstName, lastName
    ORDER BY lastName


""", conn)

,firstName,lastName,products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [36]:
pd.read_sql("""

SELECT emp.firstName, emp.lastName, SUM(quantityOrdered) AS products_sold
    FROM employees AS emp
    JOIN customers AS c
    ON emp.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders
        USING(customerNumber)
    JOIN orderdetails
        USING(orderNumber)
    GROUP BY firstName, lastName
    HAVING products_sold > 200
    ORDER BY lastName
   


""", conn)

,firstName,lastName,products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!